# 1.Importing Kaggle API and Downloading Dataset

In [1]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"atharvaumbarkar","key":"1a287e5c666a157fd3a13c5edd0cfdc3"}'}

In [2]:
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [3]:
!kaggle datasets download -d CooperUnion/anime-recommendations-database

 36% 9.00M/25.0M [00:00<00:00, 43.6MB/s]
100% 25.0M/25.0M [00:00<00:00, 83.4MB/s]


In [4]:
!unzip anime-recommendations-database.zip
!rm anime-recommendations-database.zip

Archive:  anime-recommendations-database.zip
  inflating: anime.csv               
  inflating: rating.csv              


# Importing neccessary libraries

In [63]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

# Preprocessing Data

In [64]:
anime_df = pd.read_csv('anime.csv')
rating_df = pd.read_csv('rating.csv')

In [65]:
anime_df

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266
...,...,...,...,...,...,...,...
12289,9316,Toushindai My Lover: Minami tai Mecha-Minami,Hentai,OVA,1,4.15,211
12290,5543,Under World,Hentai,OVA,1,4.28,183
12291,5621,Violence Gekiga David no Hoshi,Hentai,OVA,4,4.88,219
12292,6133,Violence Gekiga Shin David no Hoshi: Inma Dens...,Hentai,OVA,1,4.98,175


In [66]:
rating_df

,user_id,anime_id,rating
0,1,20,-1
1,1,24,-1
2,1,79,-1
3,1,226,-1
4,1,241,-1
...,...,...,...
7813732,73515,16512,7
7813733,73515,17187,9
7813734,73515,22145,10
7813735,73516,790,9


In [67]:
anime_df.fillna(0, inplace=True)

In [68]:
anime_df.episodes = anime_df.episodes.apply(lambda x: 0 if x=='Unknown' else int(x))
anime_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12294 entries, 0 to 12293
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   anime_id  12294 non-null  int64  
 1   name      12294 non-null  object 
 2   genre     12294 non-null  object 
 3   type      12294 non-null  object 
 4   episodes  12294 non-null  int64  
 5   rating    12294 non-null  float64
 6   members   12294 non-null  int64  
dtypes: float64(1), int64(3), object(3)
memory usage: 672.5+ KB


In [69]:
anime_df.type.unique()

array(['Movie', 'TV', 'OVA', 'Special', 'Music', 'ONA', 0], dtype=object)

In [70]:
def sepGenres(x):
  x = [i.strip() for i in str(x).split(',')]
  return x

In [71]:
anime_df.genre = anime_df.genre.apply(sepGenres)
anime_df

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"[Drama, Romance, School, Supernatural]",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"[Action, Adventure, Drama, Fantasy, Magic, Mil...",TV,64,9.26,793665
2,28977,Gintama°,"[Action, Comedy, Historical, Parody, Samurai, ...",TV,51,9.25,114262
3,9253,Steins;Gate,"[Sci-Fi, Thriller]",TV,24,9.17,673572
4,9969,Gintama&#039;,"[Action, Comedy, Historical, Parody, Samurai, ...",TV,51,9.16,151266
...,...,...,...,...,...,...,...
12289,9316,Toushindai My Lover: Minami tai Mecha-Minami,[Hentai],OVA,1,4.15,211
12290,5543,Under World,[Hentai],OVA,1,4.28,183
12291,5621,Violence Gekiga David no Hoshi,[Hentai],OVA,4,4.88,219
12292,6133,Violence Gekiga Shin David no Hoshi: Inma Dens...,[Hentai],OVA,1,4.98,175


In [72]:
temp = pd.get_dummies(anime_df['type'])
temp.drop(columns=[0], inplace=True)
anime_df.drop(columns=['type', 'anime_id'],inplace=True)
anime_df = anime_df.join(temp)
anime_df

,name,genre,episodes,rating,members,Movie,Music,ONA,OVA,Special,TV
0,Kimi no Na wa.,"[Drama, Romance, School, Supernatural]",1,9.37,200630,1,0,0,0,0,0
1,Fullmetal Alchemist: Brotherhood,"[Action, Adventure, Drama, Fantasy, Magic, Mil...",64,9.26,793665,0,0,0,0,0,1
2,Gintama°,"[Action, Comedy, Historical, Parody, Samurai, ...",51,9.25,114262,0,0,0,0,0,1
3,Steins;Gate,"[Sci-Fi, Thriller]",24,9.17,673572,0,0,0,0,0,1
4,Gintama&#039;,"[Action, Comedy, Historical, Parody, Samurai, ...",51,9.16,151266,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...
12289,Toushindai My Lover: Minami tai Mecha-Minami,[Hentai],1,4.15,211,0,0,0,1,0,0
12290,Under World,[Hentai],1,4.28,183,0,0,0,1,0,0
12291,Violence Gekiga David no Hoshi,[Hentai],4,4.88,219,0,0,0,1,0,0
12292,Violence Gekiga Shin David no Hoshi: Inma Dens...,[Hentai],1,4.98,175,0,0,0,1,0,0


In [73]:
def genreOnehoter(df):
  for i, row in df.iterrows():
    for genre in row['genre']:
      if genre not in anime_df.columns:
        anime_df[genre] = 0
        anime_df.at[i, genre] = 1
      else:
        anime_df.at[i, genre] = 1
  return df

In [74]:
anime_df = genreOnehoter(anime_df)
anime_df

,name,genre,episodes,rating,members,Movie,Music,ONA,OVA,Special,TV,Drama,Romance,School,Supernatural,Action,Adventure,Fantasy,Magic,Military,Shounen,Comedy,Historical,Parody,Samurai,Sci-Fi,Thriller,Sports,Super Power,Space,Slice of Life,Mecha,Mystery,Seinen,Martial Arts,Vampire,Shoujo,Horror,Police,Psychological,Demons,Ecchi,Josei,Shounen Ai,Game,Dementia,Harem,Cars,Kids,Shoujo Ai,0,Hentai,Yaoi,Yuri
0,Kimi no Na wa.,"[Drama, Romance, School, Supernatural]",1,9.37,200630,1,0,0,0,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Fullmetal Alchemist: Brotherhood,"[Action, Adventure, Drama, Fantasy, Magic, Mil...",64,9.26,793665,0,0,0,0,0,1,1,0,0,0,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Gintama°,"[Action, Comedy, Historical, Parody, Samurai, ...",51,9.25,114262,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Steins;Gate,"[Sci-Fi, Thriller]",24,9.17,673572,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Gintama&#039;,"[Action, Comedy, Historical, Parody, Samurai, ...",51,9.16,151266,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12289,Toushindai My Lover: Minami tai Mecha-Minami,[Hentai],1,4.15,211,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
12290,Under World,[Hentai],1,4.28,183,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
12291,Violence Gekiga David no Hoshi,[Hentai],4,4.88,219,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
12292,Violence Gekiga Shin David no Hoshi: Inma Dens...,[Hentai],1,4.98,175,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


In [75]:
anime_df.drop(columns=['genre', '0'],inplace=True)

In [76]:
anime_names = list(anime_df.name)
temp = []
word_index = {}
index_word = {}

for i, val in enumerate(anime_names):
  word_index[val] = i
  index_word[i] = val
  temp.append(i)

anime_df['name'] = temp
anime_df

,name,episodes,rating,members,Movie,Music,ONA,OVA,Special,TV,Drama,Romance,School,Supernatural,Action,Adventure,Fantasy,Magic,Military,Shounen,Comedy,Historical,Parody,Samurai,Sci-Fi,Thriller,Sports,Super Power,Space,Slice of Life,Mecha,Mystery,Seinen,Martial Arts,Vampire,Shoujo,Horror,Police,Psychological,Demons,Ecchi,Josei,Shounen Ai,Game,Dementia,Harem,Cars,Kids,Shoujo Ai,Hentai,Yaoi,Yuri
0,0,1,9.37,200630,1,0,0,0,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,64,9.26,793665,0,0,0,0,0,1,1,0,0,0,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2,51,9.25,114262,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,3,24,9.17,673572,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,4,51,9.16,151266,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12289,12289,1,4.15,211,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
12290,12290,1,4.28,183,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
12291,12291,4,4.88,219,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
12292,12292,1,4.98,175,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


In [77]:
# anime_df.drop(columns=['name'],inplace=True)

In [78]:
def ratingRounder(x):
  return round(x*10)

In [79]:
anime_df.rating = anime_df.rating.apply(ratingRounder)
anime_df

,name,episodes,rating,members,Movie,Music,ONA,OVA,Special,TV,Drama,Romance,School,Supernatural,Action,Adventure,Fantasy,Magic,Military,Shounen,Comedy,Historical,Parody,Samurai,Sci-Fi,Thriller,Sports,Super Power,Space,Slice of Life,Mecha,Mystery,Seinen,Martial Arts,Vampire,Shoujo,Horror,Police,Psychological,Demons,Ecchi,Josei,Shounen Ai,Game,Dementia,Harem,Cars,Kids,Shoujo Ai,Hentai,Yaoi,Yuri
0,0,1,94,200630,1,0,0,0,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,64,93,793665,0,0,0,0,0,1,1,0,0,0,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2,51,92,114262,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,3,24,92,673572,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,4,51,92,151266,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12289,12289,1,42,211,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
12290,12290,1,43,183,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
12291,12291,4,49,219,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
12292,12292,1,50,175,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


In [80]:
from sklearn.preprocessing import StandardScaler

In [81]:
members_scaler = StandardScaler()
episodes_scaler = StandardScaler()

temp = np.array(anime_df['members']).reshape(-1,1)
anime_df.members = members_scaler.fit_transform(temp)

temp = np.array(anime_df['episodes']).reshape(-1,1)
anime_df.episodes = episodes_scaler.fit_transform(temp)

anime_df

,name,episodes,rating,members,Movie,Music,ONA,OVA,Special,TV,Drama,Romance,School,Supernatural,Action,Adventure,Fantasy,Magic,Military,Shounen,Comedy,Historical,Parody,Samurai,Sci-Fi,Thriller,Sports,Super Power,Space,Slice of Life,Mecha,Mystery,Seinen,Martial Arts,Vampire,Shoujo,Horror,Police,Psychological,Demons,Ecchi,Josei,Shounen Ai,Game,Dementia,Harem,Cars,Kids,Shoujo Ai,Hentai,Yaoi,Yuri
0,0,-0.238677,94,3.330241,1,0,0,0,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,1.123326,93,14.148406,0,0,0,0,0,1,1,0,0,0,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2,0.842278,92,1.754713,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,3,0.258562,92,11.957666,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,4,0.842278,92,2.429742,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12289,12289,-0.238677,42,-0.325809,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
12290,12290,-0.238677,43,-0.326320,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
12291,12291,-0.173820,49,-0.325663,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
12292,12292,-0.238677,50,-0.326466,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


In [82]:
from sklearn.utils import shuffle

In [83]:
anime_df = shuffle(anime_df)
anime_df

,name,episodes,rating,members,Movie,Music,ONA,OVA,Special,TV,Drama,Romance,School,Supernatural,Action,Adventure,Fantasy,Magic,Military,Shounen,Comedy,Historical,Parody,Samurai,Sci-Fi,Thriller,Sports,Super Power,Space,Slice of Life,Mecha,Mystery,Seinen,Martial Arts,Vampire,Shoujo,Horror,Police,Psychological,Demons,Ecchi,Josei,Shounen Ai,Game,Dementia,Harem,Cars,Kids,Shoujo Ai,Hentai,Yaoi,Yuri
6223,6223,-0.238677,62,-0.326794,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
10460,10460,0.301800,40,-0.328965,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
49,49,-0.000867,86,7.010590,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
5983,5983,0.690944,63,-0.326411,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
5781,5781,-0.238677,64,-0.293812,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12064,12064,-0.217058,56,-0.316177,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1
11252,11252,-0.217058,70,-0.144027,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
9899,9899,-0.195439,58,-0.328053,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
377,377,-0.238677,81,0.046657,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [84]:
X_train = anime_df.drop(columns=['rating'])
y_train = anime_df['rating']
X_train.shape

(12294, 51)

In [85]:
X_train = np.array(X_train)
y_train = np.array(y_train)
y_train

array([62, 40, 86, ..., 58, 81, 76])

# Creating Model

## Based on Anime Name

In [86]:
from keras.layers import  Input, Dense, Embedding, Flatten, Dot, Softmax
from keras.models import Model
import tensorflow as tf

In [87]:
anime_vocab_size = X_train.shape[0]
input_length = X_train.shape[1]
embedding_dims = 200
classes = 101

In [88]:
def createModel(anime_vocab_size, input_length, embedding_dims, classes):

  X = Input([input_length])
  X1 = Embedding(anime_vocab_size + 1, embedding_dims, name= 'Anime_Embeddings')(X)
  X1 = Flatten()(X1)
  X1 = Dense(64, activation = 'relu')(X1)
  X1 = Dense(32, activation = 'relu')(X1)
  X1 = Dense(classes, activation = 'softmax')(X1)

  model = Model(X, X1)

  return model

In [89]:
anime_model = createModel(anime_vocab_size, input_length, embedding_dims, classes)

In [90]:
anime_model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

In [91]:
anime_model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 51)                0         
_________________________________________________________________
Anime_Embeddings (Embedding) (None, 51, 200)           2459000   
_________________________________________________________________
flatten_2 (Flatten)          (None, 10200)             0         
_________________________________________________________________
dense_4 (Dense)              (None, 64)                652864    
_________________________________________________________________
dense_5 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_6 (Dense)              (None, 101)               3333      
Total params: 3,117,277
Trainable params: 3,117,277
Non-trainable params: 0
_________________________________________________

In [92]:
history = anime_model.fit(X_train, y_train, epochs = 50, batch_size = 10)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/50
12294/12294 [==============================] - 10s 783us/step - loss: 3.6704 - accuracy: 0.0523
Epoch 2/50
12294/12294 [==============================] - 9s 757us/step - loss: 3.0168 - accuracy: 0.1569
Epoch 3/50
12294/12294 [==============================] - 9s 754us/step - loss: 1.4438 - accuracy: 0.5325
Epoch 4/50
12294/12294 [==============================] - 9s 748us/step - loss: 0.5693 - accuracy: 0.8219
Epoch 5/50
12294/12294 [==============================] - 9s 748us/step - loss: 0.1382 - accuracy: 0.9664
Epoch 6/50
12294/12294 [==============================] - 9s 746us/step - loss: 0.0436 - accuracy: 0.9917
Epoch 7/50
12294/12294 [==============================] - 9s 746us/step - loss: 0.0195 - accuracy: 0.9966
Epoch 8/50
12294/12294 [==============================] - 9s 746us/step - loss: 0.0140 - accuracy: 0.9973
Epoch 9/50
12294/12294 [==============================] - 9s 747us/step - loss: 0.0117 - accuracy: 0.9975
Epoch 10/50
12294/12294 [====================

In [93]:
embed_layer = anime_model.layers[1]
anime_embeds = embed_layer.get_weights()[0]
anime_embeds.shape

(12295, 200)

In [94]:
anime_embeddings = {}
for i in range(anime_vocab_size):
  anime_embeddings[index_word[i]] = anime_embeds[i]

anime_embeddings['Kimi no Na wa.']

array([-3.95928882e-03,  2.71042739e-03, -1.13692544e-02,  9.05677304e-03,
       -5.88316517e-03, -3.60482256e-03,  6.04433240e-04, -2.03734636e-03,
       -1.86206563e-03,  5.79984975e-04, -3.50117800e-03,  7.19499221e-05,
        7.09674954e-02,  1.40965264e-02, -6.50804257e-03,  8.20912141e-03,
       -3.79319442e-03, -5.35103446e-03, -3.51642910e-03, -4.28363215e-03,
       -9.76323907e-04, -6.39593825e-02,  7.92452920e-06, -6.73356326e-03,
        1.85025972e-03, -6.62298175e-03, -1.62532777e-02, -8.47385824e-03,
        2.66551389e-03,  1.21913978e-03, -1.43271787e-02, -4.12782235e-03,
       -1.08463457e-02, -8.35167943e-04, -6.04551286e-03,  3.26674664e-03,
        1.25940051e-02, -1.34854007e-03,  2.86605861e-03,  1.03701279e-02,
       -1.36177063e-01, -8.89902562e-03, -1.50696293e-03,  4.54310328e-03,
       -2.05876236e-03, -1.66958924e-02, -6.52614376e-03, -4.86260606e-03,
       -9.00264349e-05, -1.10617913e-02,  2.00384259e-02, -5.17930556e-03,
        8.45183793e-04,  

### Testing the predictions(not neccessary)

In [95]:
y = anime_model.predict(X_train)
y = np.argmax(y, axis=1)
for i in range(10):
  print(y[i])

62
40
86
63
64
74
65
50
47
68


# Get recommendations based on an your input anime

In [96]:
from sklearn.metrics.pairwise import cosine_similarity, cosine_distances

In [101]:
my_anime = index_word[40]
sim_list = []
for i, word in index_word.items():
  temp = cosine_similarity(anime_embeddings[my_anime].reshape(1,-1),anime_embeddings[word].reshape(1,-1))
  sim_list.append(temp)

sim_list = np.array(sim_list)
sim_list = sim_list.reshape(sim_list.shape[0])

In [102]:
recommendations = np.argsort(sim_list)[-10:]

for i in recommendations[::-1]:
  print(index_word[i])

Death Note
Kara no Kyoukai 5: Mujun Rasen
Kizumonogatari II: Nekketsu-hen
Hajime no Ippo: Rising
Mushishi Special: Hihamukage
Ping Pong The Animation
Minna no Doutoku
Natsume Yuujinchou San
Shisei Sasshin
Gintama°: Aizome Kaori-hen


# Save the Model

In [103]:
import h5py

In [104]:
anime_model.save('anime_recommender_v2.h5')

# Save the Embeddings for Visualization

In [105]:
import io

out_v = io.open('anime_vecs_v2.tsv', 'w', encoding='utf-8')
out_m = io.open('anime_meta_v2.tsv', 'w', encoding='utf-8')
for word_num in range(0, anime_vocab_size):
  word = index_word[word_num]
  embeddings = anime_embeds[word_num]
  out_m.write(word + "\n")
  out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
out_v.close()
out_m.close()

files.download('anime_vecs_v2.tsv')
files.download('anime_meta_v2.tsv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>